# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# import numpy and pandas
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np
import scipy.stats as stats



# Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [2]:
# Run this code:

pokemon = pd.read_csv('../pokemon.csv')

Let's start off by looking at the `head` function in the cell below.

In [3]:
# Your code here:
pokemon.head()


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [20]:
# Your code here:
legendario = pokemon[pokemon['Legendary'] == True]
legendario.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
156,144,Articuno,Ice,Flying,580,90,85,100,95,125,85,1,True
157,145,Zapdos,Electric,Flying,580,90,90,85,125,90,100,1,True
158,146,Moltres,Fire,Flying,580,90,100,90,125,85,90,1,True
162,150,Mewtwo,Psychic,NaN,680,106,110,90,154,90,130,1,True
163,150,MewtwoMega Mewtwo X,Psychic,Fighting,780,106,190,100,154,100,130,1,True


In [19]:
regular = pokemon[pokemon['Legendary'] == False]
regular.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [21]:
# Your code here:

legendario_mean = np.mean(legendario['Total'])

legendario_std = np.std(legendario['Total'])

print(legendario_mean)
print(legendario_std)


637.3846153846154
60.46682302213781


In [22]:
regular_mean = np.mean(regular['Total'])

regular_std = np.std(regular['Total'])

print(regular_mean)
print(regular_std)

417.21360544217686
106.68776660321996


The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [23]:
# Your code here:
from scipy.stats import ttest_ind


In [28]:
legendario_total = legendario['Total']
regular_total = regular['Total']

statistic, p_value = ttest_ind(legendario_total, regular_total, equal_var=False)


print("Estadístico de la prueba t:", statistic)
print("valor p de la prueba t:", p_value)

Estadístico de la prueba t: 25.8335743895517
valor p de la prueba t: 9.357954335957446e-47


What do you conclude from this test? Write your conclusions below.

# Your conclusions here:
Esto indica que existe una diferencia significativa en los puntos totales entre los Pokémon legendarios y no legendarios.


How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [33]:
# Your code here:

type_counts = pokemon['Type 1'].value_counts()


type_counts


Water       112
Normal       98
Grass        70
Bug          69
Psychic      57
Fire         52
Electric     44
Rock         44
Dragon       32
Ground       32
Ghost        32
Dark         31
Poison       28
Steel        27
Fighting     27
Ice          24
Fairy        17
Flying        4
Name: Type 1, dtype: int64

In [40]:
pokemon['Type 1'].value_counts().unique() 

# Me esta contando los valores unicos de los numeros no de los tipos. 

array([112,  98,  70,  69,  57,  52,  44,  32,  31,  28,  27,  24,  17,
         4])

In [44]:
len(pokemon['Type 1'].unique())
# Aqui si me cuenta el valor unico de los tipos

18

Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [46]:
# Your code here:
water_pokemon = pokemon[pokemon['Type 1'] == 'Water']
water_pokemon.head()


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43,1,False
10,8,Wartortle,Water,NaN,405,59,63,80,65,80,58,1,False
11,9,Blastoise,Water,NaN,530,79,83,100,85,105,78,1,False
12,9,BlastoiseMega Blastoise,Water,NaN,630,79,103,120,135,115,78,1,False
59,54,Psyduck,Water,NaN,320,50,52,48,65,50,55,1,False


In [49]:
no_water_pokemon = pokemon[pokemon['Type 1'] != 'Water']
no_water_pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [50]:
water_mean = np.mean(water_pokemon['Total'])
water_std = np.std(water_pokemon['Total'])

print(water_mean)
print(water_std)

430.45535714285717
112.68182832909096


In [51]:
no_water_mean = np.mean(no_water_pokemon['Total'])
no_water_std = np.std(no_water_pokemon['Total'])

print(no_water_mean)
print(no_water_std)

435.85901162790697
121.00364762501167


Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

In [52]:
# Your code here:


# hipótesis t para muestras INDEPENDIENTES!!!! asumiendo VARIANZAS IGUALES!! 

statistic, p_value = ttest_ind(water_pokemon['Total'], no_water_pokemon['Total'], equal_var=True)

# Imprimir los resultados
print("Estadístico de la prueba t:", statistic)
print("Valor p de la prueba t:", p_value)


Estadístico de la prueba t: -0.4418547448849676
Valor p de la prueba t: 0.6587140317488793


Write your conclusion below.

# Your conclusions here:
Dado que el valor p es mayor que 0.05 (NIVEL DE CONFIANZA 95% ), no podemos rechazar la hipótesis nula. Esto indica que no hay suficiente evidencia estadística para afirmar que hay una diferencia significativa en la media de los puntos totales entre los Pokémon de agua y los que no son de agua.


# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

Nuestro objetivo es ver si hay una diferencia significativa entre los puntajes de defensa y ataque de cada Pokémon. Nuestra hipótesis es que los puntajes de defensa y ataque son iguales.

In [57]:
# Your code here:

from scipy.stats import ttest_rel
defense_scores = pokemon['Defense']
attack_scores = pokemon['Attack']


In [59]:
statistic, p_value = ttest_rel(defense_scores, attack_scores)
print("Estadístico de la prueba t:", statistic)
print("Valor p de la prueba t:", p_value)

Estadístico de la prueba t: -4.325566393330478
Valor p de la prueba t: 1.7140303479358558e-05


Describe the results of the test in the cell below.

# Your conclusions here:
Existe una diferencia significativa entre los puntajes de defensa y ataque de los Pokémon.
El valor P de la prueba t es muy lejana a 0.05


In [60]:
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

diferencia significativa entre la media de defensa especial y la media de ataque especial.

In [61]:
# Your code here:
Sp_Def_scores = pokemon['Sp. Atk']
Sp_Atk_scores = pokemon['Sp. Def']


In [62]:
statistic, p_value = ttest_rel(Sp_Def_scores, Sp_Atk_scores)
print("Estadístico de la prueba t:", statistic)
print("Valor p de la prueba t:", p_value)

Estadístico de la prueba t: 0.853986188453353
Valor p de la prueba t: 0.3933685997548122


Describe the results of the test in the cell below.

# Your conclusions here:
No podemos rechazar la hipótesis nula dado que el valor p es mayor que 0.05 


As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [67]:
# Your code here:
from scipy.stats import ttest_1samp

# 1 Calcular la diferencia entre las columnas de defensa y ataque

diferencia = pokemon['Defense'] - pokemon['Attack']



In [68]:

# 2 Realizar la prueba t 
statistic, p_value = ttest_1samp(diferencia, 0)

# El segundo argumento, 0, representa el valor de referencia 
# con el cual queremos comparar la media de las diferencias. 
# Al establecerlo en cero, estamos evaluando si la media de las diferencias es significativamente diferente de cero.

print("Estadístico de la prueba t:", statistic)
print("Valor p de la prueba t:", p_value)    
    

Estadístico de la prueba t: -4.325566393330478
Valor p de la prueba t: 1.7140303479358558e-05


# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [74]:
# Your code here:

contingency_table = pd.crosstab(pokemon['Legendary'], pokemon['Type 1'] == 'Water')
contingency_table


Type 1,False,True
Legendary,,
False,627,108
True,61,4


In [73]:
contingency_table.index = ['No Legendario', 'Legendario']
contingency_table.columns = ['No Agua', 'Agua']

print(contingency_table)

               No Agua  Agua
No Legendario      627   108
Legendario          61     4


Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

La prueba de chi cuadrado se utiliza para determinar si hay una asociación significativa entre dos variables categóricas en una muestra de datos. El objetivo es evaluar si las diferencias observadas en las frecuencias observadas entre las categorías son lo suficientemente grandes como para considerarlas como evidencia en contra de la independencia entre las variables.

La hipótesis nula de la prueba de chi cuadrado establece que no hay asociación entre las variables categóricas y las diferencias observadas son el resultado del azar. Por otro lado, la hipótesis alternativa sostiene que sí hay una asociación entre las variables.

Para realizar la prueba de chi cuadrado, se sigue el siguiente proceso paso a paso:

Se crea una tabla de contingencia que muestre las frecuencias observadas de las variables categóricas. En esta tabla, las filas representan una categoría de una variable y las columnas representan una categoría de la otra variable.

Se aplica la prueba de chi cuadrado utilizando la función chi2_contingency de scipy.stats. Esta función calcula el estadístico de la prueba de chi cuadrado, el valor p correspondiente, los grados de libertad y las frecuencias esperadas bajo la hipótesis nula.

Se interpreta el resultado de la prueba. El estadístico de chi cuadrado indica cuánto difieren las frecuencias observadas de las esperadas bajo la hipótesis nula. Si el estadístico de chi cuadrado es grande, indica que hay una asociación significativa entre las variables. El valor p nos indica la probabilidad de obtener un resultado tan extremo como el observado bajo la hipótesis nula. Si el valor p es menor que un nivel de significancia predefinido (por ejemplo, 0.05), se rechaza la hipótesis nula y se concluye que hay una asociación significativa entre las variables.



In [75]:
# Your code here:
from scipy.stats import chi2_contingency

# Realizar la prueba de chi cuadrado utilizando chi2_contingency
chi2_stat, p_val, dof, expected = chi2_contingency(contingency_table)


print("Estadístico de la prueba de chi cuadrado:", chi2_stat)
print("Valor p de la prueba de chi cuadrado:", p_val)
print("Grados de libertad:", dof)
print("Frecuencias esperadas:")
print(expected)


Estadístico de la prueba de chi cuadrado: 2.9429200762850503
Valor p de la prueba de chi cuadrado: 0.08625467249550949
Grados de libertad: 1
Frecuencias esperadas:
[[632.1 102.9]
 [ 55.9   9.1]]


El resultado de la prueba de chi cuadrado ofrece información sobre la relación entre las variables en una tabla de contingencia. Veamos cada uno de los resultados obtenidos:

1. Estadístico de la prueba de chi cuadrado: El valor del estadístico de la prueba de chi cuadrado es 2.943. Este valor indica la magnitud de la diferencia entre las frecuencias observadas y esperadas en la tabla de contingencia. Cuanto mayor sea el valor del estadístico, mayor será la evidencia en contra de la hipótesis nula y mayor será la asociación entre las variables.

2. Valor p de la prueba de chi cuadrado: El valor p obtenido es 0.086. El valor p indica la probabilidad de obtener los resultados observados o resultados aún más extremos, si la hipótesis nula de independencia fuera verdadera. En este caso, el valor p es mayor que el nivel de significancia comúnmente utilizado (como 0.05). Por tanto, no hay suficiente evidencia para rechazar la hipótesis nula de independencia. Esto sugiere que no hay una asociación significativa entre las variables "Legendary" y "Type 1" en los datos.

3. Grados de libertad: Los grados de libertad son 1 en este caso. Los grados de libertad se calculan como (filas - 1) * (columnas - 1) en una tabla de contingencia. En este caso, hay 2 filas y 2 columnas, por lo que (2 - 1) * (2 - 1) = 1.

4. Frecuencias esperadas: La tabla de frecuencias esperadas muestra los valores que se esperarían en cada célula de la tabla de contingencia si la hipótesis nula de independencia fuera verdadera. Estos valores son calculados por la prueba de chi cuadrado. En este caso, los valores esperados son [[632.1, 102.9],[55.9, 9.1]]. Nos proporciona una idea de qué valores esperaríamos si las variables fueran independientes.

En resumen, los resultados sugieren que no hay suficiente evidencia para rechazar la hipótesis nula de independencia entre las variables "Legendary" y "Type 1". Esto implica que no hay una asociación significativa entre si un Pokémon es legendario o no y si su tipo 1 es agua o no.


Based on a 95% confidence, should we reject the null hypothesis?

In [19]:
# Your answer here:

